In [1]:
import sys
import os
import csv
from tqdm import tqdm_notebook
import pandas
import numpy
from pandas.tseries.holiday import USFederalHolidayCalendar
from statsmodels.tsa.arima_model import ARIMA
import sklearn.model_selection

import warnings
warnings.simplefilter('once', category=UserWarning)

os.chdir('/home/chlang/Projects/prognosec')

sys.path.append('/home/chlang/Projects/prognosec/prognosec')

%load_ext autoreload
%autoreload 1
%aimport models
%aimport timeseries

In [25]:
with open('data/data-1574048454921.csv', 'r', encoding='utf-8') as f:
    csvreader = csv.DictReader(f, delimiter=',')

    ts_ed = [row for row in csvreader]

    ts_ed = pandas.DataFrame.from_dict(ts_ed).drop(['frequency', 'intraperiod'], axis=1)
    ts_ed['date'] = pandas.to_datetime(ts_ed['date'], format='%Y-%m-%d', utc=True)
    ts_ed = ts_ed.set_index('date').drop('secid', 1).apply(pandas.to_numeric)

    ts_ed = timeseries.Timeseries(ts_ed, 'adj_close', timeseries.CAL_NYSE)
    
with open('data/data-1574048640489.csv', 'r', encoding='utf-8') as f:
    csvreader = csv.DictReader(f, delimiter=',')

    ts_aapl = [row for row in csvreader]

    ts_aapl = pandas.DataFrame.from_dict(ts_aapl).drop(['frequency', 'intraperiod'], axis=1)
    ts_aapl['date'] = pandas.to_datetime(ts_aapl['date'], format='%Y-%m-%d', utc=True)
    ts_aapl = ts_aapl.set_index('date').drop('secid', 1).apply(pandas.to_numeric)

#     ts_aapl = timeseries.Timeseries(ts_aapl, 'adj_close', timeseries.CAL_NASDAQ)

ValueError: No series can have NaN/Inf

In [34]:
ts = ts_ed
primary_series = 'adj_close'
calendar = timeseries.CAL_NYSE

In [32]:
if calendar is not None:
    index_name = ts.index.name
    timeseries_dates = calendar.schedule(ts.index.min(), ts.index.max())
    ts = ts.reindex(index=timeseries_dates.index)
    ts.index.name = index_name

In [38]:
timeseries_dates.index

DatetimeIndex(['2015-08-06', '2015-08-07', '2015-08-10', '2015-08-11',
               '2015-08-12', '2015-08-13', '2015-08-14', '2015-08-17',
               '2015-08-18', '2015-08-19',
               ...
               '2019-10-18', '2019-10-21', '2019-10-22', '2019-10-23',
               '2019-10-24', '2019-10-25', '2019-10-28', '2019-10-29',
               '2019-10-30', '2019-10-31'],
              dtype='datetime64[ns]', name='date', length=1068, freq='C')

In [11]:
fit_results = list()
for an_order in tqdm_notebook(orders):
    r = models.cv_timeseries(ts_aapl, models.Arima, order=an_order)
    fit_results.append(r)
    
final_fit_result = pandas.concat(fit_results)

/home/chlang/anaconda3/envs/promometheus/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/home/chlang/anaconda3/envs/promometheus/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/home/chlang/anaconda3/envs/promometheus/lib/python3.7/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:221: RuntimeWarning:

divide by zero encountered in true_divide

/home/chlang/anaconda3/envs/promometheus/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning:

Inverting hessian failed, no bse or cov_params available

/home/chlang/anaconda3/envs/promometheus/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning:

Inverting hessian failed, no bse or cov_params available

